<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/tobzed/Lab-2/tedwards_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 2: Direct Methods**
**Tobias Edwards**

# **Abstract**

This lab focussed on implementing the following:

1. QR Factorization of matrix A
2. A direct solver for Ax =b
3. Least squares method for overdetermined systems
4. QR eigenvalue algorithm



#**About the code**

In [0]:
# This code is written for Lab 2 in DD2363 Methods in Scientific Computing
# Course given by the Royal Institute of Technology in Stockholm, KTH
# Code by Tobias Edwards (tedwards@kth.se), Spring 2019

# **Set up environment**

These are the environment variables, make sure to run this code before trying any code below! 

In [0]:
# Load neccessary modules.
from google.colab import files

import unittest
import time
import numpy as np

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

$QR$ factorization is used to represent matrices in such a way that is is easier to numerically calculate matrix inverses, for instance. $Q$ is a orthogonal matrix such that $Q^T$ = $Q^{-1}$.  Backward substitution is a iterative method used to solve the system $Rx = y$ where $R$ is an upper triangular matrix. Least squares method is a way to solve overdetermined systems that do not have exact solutions. In this case, a solution to an overdetermined system $Ax =b$, $A \in R^{m\times n}, x \in R^{n}, b \in R^{m}$, is $x$ such that $||Ax-b|| \leq ||Ay-b||, \forall y \in R^n$.



# **Results**

In [0]:
#QR factorization

def gs(A):
    #gram-schmidt method for creating a orthonormal matrix Q and upper triangular matrix R from A
    rows,cols = A.shape
    Q = np.zeros((rows,cols))
    R = np.zeros((cols,cols))
    for j in range(cols):
        vj = A[:,j]
        if j > 0: # if j == 0, we don't need to modify the direction of the corresponding vector
            for i in range(j):
                R[i,j] = np.inner(Q[:,i],A[:,j])
                vj = np.subtract(vj,R[i,j]*Q[:,i])
        R[j,j] = np.linalg.norm(vj)
        Q[:,j] = vj/R[j,j]
    return Q, R
  
A = np.array( [[1,2,0], [4,0,2], [100,0,1]] )
Q,R = gs(A)
print("Q = ", Q, "R = ", R, "A = ", A, "QR = ", Q.dot(R), sep="\n\n")

In [0]:
def backward_substitution(R,b):
    # R is an upper triangular matrix
    x = np.zeros(b.shape)
    n = R.shape[0]
    x[n-1] = b[n-1] / R[n-1,n-1]
    # iterate in reverse order to find x_i = b_i - sum(R_ij*x_j)/R_ii ,for j in [i+1,n)
    for i in range(n-2,-1,-1):
        x[i] = (b[i]-sum([R[i,j]*x[j] for j in range(i+1,n)]))/R[i,i]
    return x
  
def direct_solver(A,b):
    # Ax = b <=> QRx = b <=> Rx = Q^(⁻1)b = Q^(T)b
    (Q,R) = gs(A)
    # use backward backward_substitution to solve this "new" equation system
    return backward_substitution(R,Q.transpose().dot(b))

A = np.array( [[1,2,0], [4,0,2], [100,0,1]] )
b = np.array( [3,-2,2] )
x = direct_solver(A,b)
print("A = ", A, "b = ", b, "x = ", x, sep="\n\n")
 

In [0]:
# least squares method

def least_squares(A,b):
    new_A = np.matmul(np.transpose(A),A)
    # Ax = b is overdetermined
    # normal equations: A^(T)Ax = A^(T)b
    new_b = np.transpose(A).dot(b)
    return direct_solver(new_A, new_b)

A = np.array([[4,2], [-3,9], [22,1]])
b = np.array([2,7,-1])
x = least_squares(A,b)
print("A = ", A, "b = ", b, "x = ", x, sep="\n\n")

In [0]:
# this finds the largest eigenvalue lamda_1 and corresponding eigenvector v_1 for A
def power_iteration(A):
    v_1 = np.random.rand(A.shape[0])
    v_1 *= 1/np.linalg.norm(v_1)
    lamda_1 = 0
    for k in range(100):
        w = A.dot(v_1)
        v_1 = w/np.linalg.norm(w)
        lamda_1 = np.inner(v_1, A.dot(v_1))
    return lamda_1, v_1

# **Testing**

In [0]:
 # class for testing methods in lab 2 using unittest
  
class Lab2FunctionsTest(unittest.TestCase):

    def test_qr_factorization(self):
        # randomly generate a square matrix
        A = np.random.rand(3,3)
        Q,R = gs(A)
        Qt = np.transpose(Q)
        # tranpose(Q) = inverse(Q) -> tranpose(Q) * Q = I
        # so Frobenius norm of : tranpose(Q) * Q - I = 0
        self.assertTrue(np.linalg.norm(Qt.dot(Q)-np.identity(3)) < 0.000001)
        # A = QR -> QR - A = 0-matrix -> Frobenius norm of QR-A = 0
        self.assertTrue(np.linalg.norm(Q.dot(R)-A) < 0.000001)

    def test_direct_solver(self):
        A = np.random.rand(3,3)
        b = np.random.rand(3)
        x = direct_solver(A,b)
        # if x is a solution to Ax = b, then Ax-b = 0
        self.assertTrue(np.linalg.norm(A.dot(x)-b) < 0.0000001)

        # pre calculated test
        A2 = np.array([[1,2],[4,-2]])
        y = np.array([1.4,0.8])
        b2 = np.array([3,4])
        x2 = direct_solver(A2,b2)
        # assert that the numerical solution x2 is not far from the "exact" solution y2 |x-u| < eps
        self.assertTrue(np.linalg.norm(x2-y) < 0.0000001)

    def test_least_squares(self):
        A = np.array([[1,2], [4,2], [-1,0]])
        b = np.array([4,-2,3])
        x = least_squares(A,b)
        # x_np is numpy's least squares solution (np.linalg.lstsq)
        x_np = np.linalg.lstsq(A,b,rcond=-1)[0]
        self.assertTrue(np.linalg.norm(x-x_np) < 0.00000001)

        # A test for a randomized overdetermined system
        A_rand = np.random.rand(10,4)
        b_rand = np.random.rand(10)
        x_rand = least_squares(A_rand, b_rand)
        x_np_rand = np.linalg.lstsq(A_rand,b_rand,rcond=-1)[0]
        self.assertTrue(np.linalg.norm(x_rand-x_np_rand) < 0.00000001)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.034s

OK


# **Discussion**

This lab was slightly more challening than the first lab. I could not figure out how to generate all Eigenvectors. Using the QR algorithm I was able to construct a similiar matrix to A that had A's Eigenvalues on its diagonal. But apart from the Eigenvalue problems the lab was good and interesting. 